In [1]:
import numpy as np
import pandas as pd
import h5py as h5
import importlib
import seaborn as sns
sns.set(style = 'ticks')
import configparser
from scipy import sparse

In [35]:
!ls lisa2/

bin  docs  LICENSE  lisa  MANIFEST.in  README.md  setup.py


In [37]:
!cp lisa2/lisa/gene_selection.py ./

In [2]:
from gene_selection import Gene, GeneSet

In [3]:
import utils

# Define transformation functions

In [50]:
def define_geneset(config, species, genelist_h5, tad_data_file):
    
    with h5.File(genelist_h5, 'r') as data:
        gene_list = data['RefSeq'][...].astype(str)
    
    all_genes = GeneSet()

    for geneline in gene_list:
        chrom, start, end, _id, symbol = geneline.split(':')
        newgene = Gene(chrom, start, end, [symbol, _id])
        all_genes.add_gene(newgene)
        
    tad_data = pd.read_csv(tad_data_file, sep = '\t', encoding = 'latin')
    tad_data['domain'] = (tad_data.k4me3_order_cluster.astype(str) + ',' + tad_data.tad_order_cluster.astype(str)).values
    tad_domains = tad_data.set_index('geneName').domain
        
    for gene_name, tad_domain in tad_domains.items():
        try:
            for gene in all_genes.genes_by_name[gene_name]:
                gene.tad_domain = tad_domain
        except KeyError:
            pass
        
    with open(config.get('genes','master_gene_list').format(package_path = './', species = species), 'w') as genelist:
        genelist.write(str(all_genes))
                
    return all_genes

def calc_rp_map(config, species, bin_regions_file, geneset):

    bin_regions = pd.read_csv(bin_regions_file, sep = '\t', header = None)
    bin_regions.columns = ['chrom','start','end','bin_num']
    bin_regions.bin_num -= 1
    bin_groups = bin_regions.groupby('chrom')

    loadingbar = utils.LoadingBar('Calculating RP matrix', len(geneset), length = 40)

    indices, rp_vals, gene_locs = [],[],[]

    for chrom, bins in bin_groups:
        bin_intervals = np.concatenate([[0], bins.end.values])
        genes_on_chrom = geneset.get_genes_by_chrom(chrom)
        for gene in genes_on_chrom:
            rp, bin_indices = gene.get_RP_signature(bin_intervals, bins.bin_num.values)
            rp_vals.append(rp)
            indices.append(bin_indices)
            gene_locs.append(gene.get_location())
            print('\r', loadingbar, end = '')
            
    rp_map = utils.ragged_array_to_sparse_matrix(indices, rp_vals, 
                int(config.get('lisa_params','num_bins_{}'.format(species)))).transpose()
    
    sparse.save_npz(config.get('RP_map', 'matrix').format(package_path = './', species = species), rp_map)
    
    with open(config.get('genes','gene_locs').format(package_path = './', species = species), 'w') as f:
        f.write('\n'.join(gene_locs))
    
    return rp_map, gene_locs

def read_metadata(metadata_file):
    metadata = pd.read_csv(metadata_file, sep = '\t', encoding = "ISO-8859-1").set_index('id')
    metadata.index = metadata.index.astype(np.str)
    
    try:
        metadata['qc']
    except KeyError:
        metadata['qc'] = 1
        metadata['factor'] = metadata.index        
        
    return metadata

def save_RP_matrix(h5_object, config, technology, bins_matrix, rp_map, gene_locs, dataset_ids):
    
    print('\tCalculating RP matrix ...')
    #genes x bins \dot bins * datasets = genes * datasets
    rp_matrix = rp_map.dot(bins_matrix)
    
    if isinstance(rp_matrix, (sparse.csr_matrix, sparse.csc_matrix)):
        rp_matrix = np.array(rp_matrix.todense()).astype(np.float64)
    
    print('\tSaving RP matrix ...')
    
    #del h5_object[config['accessibility_assay']['reg_potential_gene_locs'].format(technology = technology)]
    h5_object.create_dataset(
        config['accessibility_assay']['reg_potential_gene_locs'].format(technology = technology),
        data = np.array(gene_locs).astype('S')
    )
    
    #del h5_object[config['accessibility_assay']['reg_potential_dataset_ids'].format(technology = technology)]
    h5_object.create_dataset(
        config['accessibility_assay']['reg_potential_dataset_ids'].format(technology = technology),
        data = np.array(dataset_ids).astype('S')
    )
    
    #del h5_object[config['accessibility_assay']['reg_potential_matrix'].format(technology = technology)]
    h5_object.create_dataset(
        config['accessibility_assay']['reg_potential_matrix'].format(technology = technology),
        data = rp_matrix
    )

def index_binned_reads(*,h5_save_file, config, technology, rp_map, gene_locs, binned_reads_h5_file, metadata_file):
    
    print('\tReading metadata ...')
    metadata = read_metadata(metadata_file)
    qc_status = metadata.qc
    
    print('\tReading binned reads data ...')

    with h5.File(binned_reads_h5_file, 'r') as binned_reads:

        dataset_ids = binned_reads['IDs'][...].astype(str)

        dataset_subset = qc_status[dataset_ids].astype(np.bool).values

        reads_matrix = binned_reads['OrderCount'][:, dataset_subset]
        
    dataset_ids = dataset_ids[dataset_subset]
    
    reads_matrix = reads_matrix
    
    num_reads = reads_matrix.sum(axis = 0)
    
    alpha = 2.0
    bounds = (num_reads.mean() - alpha*num_reads.std(), num_reads.mean() + alpha*num_reads.std())

    normal_samples = np.logical_and(bounds[0] <= num_reads, num_reads <= bounds[1])
    
    reads_matrix = reads_matrix[:,normal_samples]
    dataset_ids = dataset_ids[normal_samples]
    
    reads_matrix = reads_matrix * 1e8 / reads_matrix.sum(axis = 0)
    
    dataset_ids = np.array([id_.split('_')[0] for id_ in dataset_ids])
    
    with h5.File(h5_save_file, 'a') as h5_object:
    
        save_RP_matrix(h5_object, config, technology, reads_matrix, rp_map, gene_locs, dataset_ids)

        loading_bar = utils.LoadingBar('\tWriting ID-indexed data', dataset_subset.sum(), 20)

        for i, dataset_id in enumerate(dataset_ids):
            print('\r', loading_bar, end = '')
            #del h5_object[config['accessibility_assay']['binned_reads'].format(technology = technology, dataset_id = dataset_id)]
            try: 
                h5_object.create_dataset(
                    config['accessibility_assay']['binned_reads'].format(technology = technology, dataset_id = dataset_id),
                    data = reads_matrix[:, i]
                )
            except OSError as err:
                print('Error saving dataset #{}: {}'.format(str(dataset_id), str(err)))

def reformat_TF_hits(*,h5_save_file, config, species, technology, rp_map, gene_locs, factor_binding_h5, window_converter_file, metadata_file, offset = 0):
    
    print('\tReading metadata ...')
    metadata = read_metadata(metadata_file)
    
    window_converter = np.load(window_converter_file)
    
    with h5.File(factor_binding_h5, 'r') as tf_hits:

        num_samples = len(list(tf_hits.keys()))
        
        loading_bar = utils.LoadingBar('\tCollecting binding data', num_samples, 20)
        
        peak_list, values, dataset_ids = [],[],[]
        
        for sample in tf_hits.keys():
            print('\r',loading_bar, end = '')
            
            if not sample == 'IDs':
                
                try:
                    sample_metadata = metadata.loc[str(sample)]
                
                    if sample_metadata.qc == 1:
                        
                        factor_name = sample_metadata.factor.replace('/', '-').replace('_','-')

                        peaks = tf_hits[sample][...]
                        peaks = window_converter[peaks - offset]
                        
                        peak_list.append(peaks)
                        values.append(np.ones(len(peaks)))
                        dataset_ids.append(sample)
                        
                except OSError:
                    print('\n\tError saving data for sample {}, factor: {}'\
                          .format(str(sample), sample_metadata.factor))
                except KeyError:
                    print('\n\tError: No metadata for sample {}'.format(str(sample)))
                    
    tf_hits = utils.ragged_array_to_sparse_matrix(peak_list, values, 
                                int(config.get('lisa_params','num_bins_{}'.format(species))))
    
    with h5.File(h5_save_file, 'a') as h5_object:
        save_RP_matrix(h5_object, config, technology, tf_hits, rp_map, gene_locs, dataset_ids)
        
    sparse.save_npz(config.get('factor_binding','matrix').format(package_path = './', species = species, technology = technology), tf_hits)


In [8]:
!wget -c http://lisa.cistrome.org/cistromedb_data/lisa_v1.2_mm10.tar.gz

!mv lisa_v1.2_mm10.tar.gz old_data/

!tar -xvf old_data/lisa_v1.2_mm10.tar.gz

!mv mm10/ old_data/

--2020-09-25 10:34:14--  http://lisa.cistrome.org/cistromedb_data/lisa_v1.2_mm10.tar.gz
Resolving lisa.cistrome.org (lisa.cistrome.org)... 155.52.218.90
Connecting to lisa.cistrome.org (lisa.cistrome.org)|155.52.218.90|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21431746089 (20G) [application/x-gzip]
Saving to: ‘lisa_v1.2_mm10.tar.gz’

lisa_v1.2_mm10.tar. 100%[===================>]  19.96G   953MB/s    in 21s     

2020-09-25 10:35:29 (954 MB/s) - ‘lisa_v1.2_mm10.tar.gz’ saved [21431746089/21431746089]



In [5]:
!ls old_data/mm10

cistrome.txt
cluster_mouse
lisa_meta.xls
margeRP_DNase_mm.h5
margeRP_H3K27ac_mm.h5
mm10_100to1000window.out.npy
mm10_beta_peak5fold.h5
mm10.genome
mm10_lisa_tf_100bp_all_nonhm_nonca_peak5fold.h5
mm10_marge2_motif_100bp_99.h5
mm10_promoter_TADann_H3K4me3_enhance_k27me3_Using.xls
mm10_tf_new_beta_rp.h5
mm10.tss
mm10_window1kb.bed
mm10_window1kb_DNase.h5
mm10_window1kb_H3K27ac.h5
mm10_window1kb_tss.bed
mm_tf_new_beta_rp.h5


In [46]:
t = h5.File('./old_data/mm10/mm_tf_new_beta_rp.h5', 'r')

In [45]:
t.close()

In [ ]:
config = configparser.ConfigParser()
config.read('./lisa2/lisa/config.ini')

geneset = define_geneset(config, 'mm10', './old_data/mm10/cluster_mouse/DNase_median_for_each_cluster.h5', 
                        'old_data/mm10/mm10_promoter_TADann_H3K4me3_enhance_k27me3_Using.xls')

rp_map, gene_locs = calc_rp_map(config, 'mm10', 'old_data/mm10/mm10_window1kb.bed', geneset)

h5_name = 'lisa_data_mm10_reads_normalized.h5'

In [51]:
with h5.File('data/mm10/' + h5_name, 'w') as data:
    pass

In [54]:
index_binned_reads(h5_save_file = 'data/mm10/' + h5_name, config = config, technology = 'DNase', 
                   rp_map = rp_map, gene_locs = gene_locs, binned_reads_h5_file = './old_data/mm10/mm10_window1kb_DNase.h5',
                   metadata_file = 'old_data/mm10/lisa_meta.xls')

	Reading metadata ...
	Reading binned reads data ...
	Calculating RP matrix ...
	Saving RP matrix ...
 	Writing ID-indexed data: [===================>]

In [55]:
index_binned_reads(h5_save_file = 'data/mm10/' + h5_name, config = config, technology = 'H3K27ac', 
                   rp_map = rp_map, gene_locs = gene_locs, binned_reads_h5_file = './old_data/mm10/mm10_window1kb_H3K27ac.h5',
                   metadata_file = 'old_data/mm10/lisa_meta.xls')

	Reading metadata ...
	Reading binned reads data ...
	Calculating RP matrix ...
	Saving RP matrix ...
 	Writing ID-indexed data: [===============>    ]Error saving dataset #44619: Unable to create link (name already exists)
 	Writing ID-indexed data: [===================>]

In [52]:
reformat_TF_hits(h5_save_file = 'data/mm10/' + h5_name, config = config, species = 'mm10', technology = 'ChIP-seq',
                 rp_map = rp_map, gene_locs = gene_locs, factor_binding_h5 = 'old_data/mm10/mm10_lisa_tf_100bp_all_nonhm_nonca_peak5fold.h5',
                 window_converter_file = 'old_data/mm10/mm10_100to1000window.out.npy', 
                 metadata_file = 'old_data/mm10/lisa_meta.xls', offset = 1)

	Reading metadata ...
	Calculating RP matrix ...
	Saving RP matrix ...


In [53]:
reformat_TF_hits(h5_save_file = 'data/mm10/' + h5_name, config = config, species = 'mm10', technology = 'Motifs',
                 rp_map = rp_map, gene_locs = gene_locs, factor_binding_h5 = 'old_data/mm10/mm10_marge2_motif_100bp_99.h5',
                 window_converter_file = 'old_data/mm10/mm10_100to1000window.out.npy', 
                 metadata_file = 'old_data/mm10/cistrome.txt', offset = 0)

	Reading metadata ...
	Error: No metadata for sample TFs
	Calculating RP matrix ...
	Saving RP matrix ...


# Investigating Read Stuff

In [7]:
m = read_metadata('old_data/hg38/lisa_meta.xls')

data = h5.File('./old_data/hg38/hg38_window1kb_DNase.h5' ,'r')

dsid = data['IDs'][...].astype(str)

ds_mask = m.qc[dsid].astype(np.bool)

arr = data['OrderCount'][:, ds_mask.values]

arr_sums = arr.sum(axis = 0)

sns.distplot(arr_sums)

In [58]:
bounds

(72740.716796875, 131616.580078125)

In [53]:
selections = set(['3147',
 '45067',
 '40800',
 '44974',
 '44921',
 '44959',
 '3136',
 '58044',
 '41430',
 '3179'])

In [57]:
alpha = 2.0
bounds = (arr_sums.mean() - alpha*arr_sums.std(), arr_sums.mean() + alpha*arr_sums.std())

outliers = ~np.logical_and(bounds[0] < arr_sums, arr_sums < bounds[1])

In [60]:
outliers.sum()

79

In [61]:
selections.intersection(set(dsid[ds_mask][outliers]))

{'3179', '58044'}

In [65]:
m.loc['58044']

species        Homo sapiens
factor                DNase
factor_type              ca
cell_line               NaN
cell_type               NaN
tissue               Thymus
qc                        1
Name: 58044, dtype: object